In [ ]:
from typing import List, Optional
import sky
import os


In [ ]:
# Total Nodes, INCLUDING Head Node
num_nodes = 2

In [ ]:
workdir =  os.getcwd()
print(workdir)

In [ ]:
# The setup command.  Will be run under the working directory.
setup = '''
  pip install "ray[train]"
  pip install tqdm
  pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
  sudo mkdir ./checkpoint
  '''


In [ ]:
# The command to run.  Will be run under the working directory.
def run_fn(node_rank: int, ip_list: List[str]) -> Optional[str]:
    num_nodes = len(ip_list)
    return f"""\
    if [ "{node_rank}" == "0" ]; then
      ps aux | grep ray | grep 6379 &> /dev/null || ray start --head  --disable-usage-stats --port 6379
      sleep 5
      python train.py --num-workers {num_nodes}
    else
      sleep 5
      ps aux | grep ray | grep 6379 &> /dev/null || ray start --address {ip_list[0]}:6379 --disable-usage-stats
    fi
    """

In [ ]:
train = sky.Task(
    'train',
    setup=setup,
    workdir=workdir,
    num_nodes=num_nodes,
    run=run_fn,
)

In [ ]:
train.set_resources({
    ##### Fully specified
    sky.Resources(sky.Kubernetes(), accelerators={'T4': 1})
})

In [ ]:
sky.launch(train, cluster_name='ray-train')